In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.base import clone
from utils.config_loader import get_path_from_yaml
from sklearn.ensemble import GradientBoostingClassifier
from logic.factory_manage.original_kensyou3 import get_df


# === ユーティリティ ===
from utils.config_loader import get_path_from_yaml
from utils.get_holydays import get_japanese_holidays
from logic.factory_manage.sql import load_data_from_sqlite


In [2]:
if __name__ == "__main__":
    import datetime
    import os

    first_date = "2024-05-01"
    last_date = "2025-05-30"

    # --- データ読み込み ---
    df_raw1 = load_data_from_sqlite(first_date, last_date)
    df_raw1["伝票日付"] = pd.to_datetime(df_raw1["伝票日付"], errors="coerce")
    df_raw1["正味重量"] = pd.to_numeric(df_raw1["正味重量"], errors="coerce")
    df_raw1 = df_raw1.dropna(subset=["正味重量"])
    df_raw1 = df_raw1[["伝票日付", "正味重量", "品名"]]

    print(f"データ件数(df_raw): {len(df_raw1)}")
    print(df_raw1["伝票日付"].min(), df_raw1["伝票日付"].max())


    # --- 差分確認用に pickle 保存 ---
    tmp_dir = "/work/tmp"
    os.makedirs(tmp_dir, exist_ok=True)

    run_id = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    df_raw1.to_pickle(f"{tmp_dir}/df_raw1_{run_id}.pkl")
    print(f"✅ df_raw1 saved as df_raw1_{run_id}.pkl")

    # --- 別ソースから df_raw2 取得 ---
    df_raw2 = get_df()
    df_raw2["伝票日付"] = pd.to_datetime(df_raw2["伝票日付"], errors="coerce")
    df_raw2["正味重量"] = pd.to_numeric(df_raw2["正味重量"], errors="coerce")
    df_raw2 = df_raw2.dropna(subset=["正味重量"])
    df_raw2 = df_raw2[["伝票日付", "正味重量", "品名"]]

    print(f"データ件数(df_raw2): {len(df_raw2)}")

    df_raw2.to_pickle(f"{tmp_dir}/df_raw2_{run_id}.pkl")
    print(f"✅ df_raw2 saved as df_raw2_{run_id}.pkl")


データ件数(df_raw): 48216
2024-05-01 00:00:00 2025-05-26 00:00:00
✅ df_raw1 saved as df_raw1_20250609_045911.pkl
2024-05-01 2025-05-26
データ件数(df_raw2): 48216
✅ df_raw2 saved as df_raw2_20250609_045911.pkl


In [3]:
df_raw1["伝票日付"] = pd.to_datetime(df_raw1["伝票日付"], errors="coerce")
df_raw2["伝票日付"] = pd.to_datetime(df_raw2["伝票日付"], errors="coerce")

df_raw1["正味重量"] = pd.to_numeric(df_raw1["正味重量"], errors="coerce")
df_raw2["正味重量"] = pd.to_numeric(df_raw2["正味重量"], errors="coerce")

### 差分確認

In [6]:
import pandas as pd

# --- ファイル名（YYYYMMDD_HHMMSSの部分は自分の保存した時刻に置き換えてね） ---
run_id = "20250609_045911"

df_raw1 = pd.read_pickle(f"/work/tmp/df_raw1_{run_id}.pkl")
df_raw2 = pd.read_pickle(f"/work/tmp/df_raw2_{run_id}.pkl")

# --- 品名の違い ---
品名1 = set(df_raw1["品名"])
品名2 = set(df_raw2["品名"])

print("✅ df_raw1 だけにある品名:")
print(品名1 - 品名2)
print()

print("✅ df_raw2 だけにある品名:")
print(品名2 - 品名1)
print()

# --- 日付の違い ---
日付1 = set(df_raw1["伝票日付"])
日付2 = set(df_raw2["伝票日付"])

print("✅ df_raw1 だけにある日付:")
print(日付1 - 日付2)
print()

print("✅ df_raw2 だけにある日付:")
print(日付2 - 日付1)
print()

# --- 件数の違い ---
print(f"✅ df_raw1 件数: {len(df_raw1)}")
print(f"✅ df_raw2 件数: {len(df_raw2)}")
print()

# --- 品名別 件数差分 ---
count1 = df_raw1["品名"].value_counts()
count2 = df_raw2["品名"].value_counts()

# 差分 DataFrame にする
count_df = pd.DataFrame({
    "df_raw1_件数": count1,
    "df_raw2_件数": count2
}).fillna(0).astype(int)

count_df["件数差(df_raw2 - df_raw1)"] = count_df["df_raw2_件数"] - count_df["df_raw1_件数"]

print("✅ 品名別 件数差分（上位20件）:")
print(count_df.sort_values("件数差(df_raw2 - df_raw1)", ascending=False).head(20))
print()

# --- 品名別 正味重量差分 ---
sum1 = df_raw1.groupby("品名")["正味重量"].sum()
sum2 = df_raw2.groupby("品名")["正味重量"].sum()

sum_df = pd.DataFrame({
    "df_raw1_正味重量": sum1,
    "df_raw2_正味重量": sum2
}).fillna(0)

sum_df["正味重量差(df_raw2 - df_raw1)"] = sum_df["df_raw2_正味重量"] - sum_df["df_raw1_正味重量"]

print("✅ 品名別 正味重量差分（上位20件）:")
print(sum_df.sort_values("正味重量差(df_raw2 - df_raw1)", ascending=False).head(20))
print()

# --- NaTを含む行の確認 ---
nat_rows = df_raw2[df_raw2["伝票日付"].isna()]
print(f"✅ df_raw2 NaT 含む行数: {len(nat_rows)}")
if len(nat_rows) > 0:
    print("✅ df_raw2 NaT 含む行サンプル:")
    print(nat_rows.head())


✅ df_raw1 だけにある品名:
set()

✅ df_raw2 だけにある品名:
set()

✅ df_raw1 だけにある日付:
set()

✅ df_raw2 だけにある日付:
set()

✅ df_raw1 件数: 48216
✅ df_raw2 件数: 48216

✅ 品名別 件数差分（上位20件）:
                 df_raw1_件数  df_raw2_件数  件数差(df_raw2 - df_raw1)
品名                                                             
混合廃棄物A                18498       18498                       0
混合廃棄物B                 5653        5653                       0
GC 軽鉄･ｽﾁｰﾙ類            3465        3465                       0
選別                     3109        3109                       0
木くず                    2979        2979                       0
軽量物系B(ｸﾞﾗｽｳｰﾙ)         1718        1718                       0
混合廃棄物(焼却物)             1667        1667                       0
ﾀﾞﾝﾎﾞｰﾙ                 786         786                       0
廃ﾌﾟﾗｽﾁｯｸ類               769         769                       0
混合廃棄物                   685         685                       0
混合廃棄物(ｿﾌｧｰ)             517         517                       0
軽量物系

In [ ]:
import pandas as pd

# --- ① 品名セット差分 ---
品名1 = set(df_raw1["品名"])
品名2 = set(df_raw2["品名"])

print("✅ df_raw1 だけにある品名:")
print(品名1 - 品名2)
print()

print("✅ df_raw2 だけにある品名:")
print(品名2 - 品名1)
print()

# --- ② 日付セット差分 ---
日付1 = set(df_raw1["伝票日付"])
日付2 = set(df_raw2["伝票日付"])

print("✅ df_raw1 だけにある日付:")
print(日付1 - 日付2)
print()

print("✅ df_raw2 だけにある日付:")
print(日付2 - 日付1)
print()

# --- ③ 品名別 件数・正味重量差分 ---
df1_summary = df_raw1.groupby("品名")["正味重量"].agg(["count", "sum"])
df2_summary = df_raw2.groupby("品名")["正味重量"].agg(["count", "sum"])

df_summary_diff = pd.DataFrame({
    "df_raw1_件数": df1_summary["count"],
    "df_raw2_件数": df2_summary["count"],
    "df_raw1_正味重量": df1_summary["sum"],
    "df_raw2_正味重量": df2_summary["sum"],
}).fillna(0)

df_summary_diff["件数差(df_raw2 - df_raw1)"] = df_summary_diff["df_raw2_件数"] - df_summary_diff["df_raw1_件数"]
df_summary_diff["正味重量差(df_raw2 - df_raw1)"] = df_summary_diff["df_raw2_正味重量"] - df_summary_diff["df_raw1_正味重量"]

print("✅ 品名別 件数差分（上位20件）:")
print(df_summary_diff.sort_values("件数差(df_raw2 - df_raw1)", ascending=False).head(20))
print()

print("✅ 品名別 正味重量差分（上位20件）:")
print(df_summary_diff.sort_values("正味重量差(df_raw2 - df_raw1)", ascending=False).head(20))
print()

# --- ④ 日付別 件数差分 ---
df1_daily_count = df_raw1.groupby("伝票日付").size()
df2_daily_count = df_raw2.groupby("伝票日付").size()

df_daily_count_diff = pd.DataFrame({
    "df_raw1_件数": df1_daily_count,
    "df_raw2_件数": df2_daily_count
}).fillna(0)

df_daily_count_diff["件数差(df_raw2 - df_raw1)"] = df_daily_count_diff["df_raw2_件数"] - df_daily_count_diff["df_raw1_件数"]

print("✅ 日付別 件数差分（件数差≠0 のもの）:")
print(df_daily_count_diff[df_daily_count_diff["件数差(df_raw2 - df_raw1)"] != 0])
print()

# --- ⑤ 日付別 合計正味重量差分 ---
df1_daily_sum = df_raw1.groupby("伝票日付")["正味重量"].sum()
df2_daily_sum = df_raw2.groupby("伝票日付")["正味重量"].sum()

df_daily_sum_diff = pd.DataFrame({
    "df_raw1_正味重量": df1_daily_sum,
    "df_raw2_正味重量": df2_daily_sum
}).fillna(0)

df_daily_sum_diff["正味重量差(df_raw2 - df_raw1)"] = df_daily_sum_diff["df_raw2_正味重量"] - df_daily_sum_diff["df_raw1_正味重量"]

print("✅ 日付別 正味重量差分（差≠0 のもの）:")
print(df_daily_sum_diff[df_daily_sum_diff["正味重量差(df_raw2 - df_raw1)"] != 0])
print()

# --- ⑥ 品名×日付ペアの件数差分（細かく比較） ---
df1_pivot = df_raw1.groupby(["伝票日付", "品名"]).size()
df2_pivot = df_raw2.groupby(["伝票日付", "品名"]).size()

df_pivot_diff = pd.DataFrame({
    "df_raw1_件数": df1_pivot,
    "df_raw2_件数": df2_pivot
}).fillna(0)

df_pivot_diff["件数差(df_raw2 - df_raw1)"] = df_pivot_diff["df_raw2_件数"] - df_pivot_diff["df_raw1_件数"]

print("✅ 品名×日付ペアの件数差分（件数差≠0 のもの）:")
print(df_pivot_diff[df_pivot_diff["件数差(df_raw2 - df_raw1)"] != 0])
print()

# --- ⑦ df_raw1 / df_raw2 件数確認 ---
print(f"✅ df_raw1 件数: {len(df_raw1)}")
print(f"✅ df_raw2 件数: {len(df_raw2)}")
print()

# --- ⑧ df_raw2 NaT 行確認 ---
nat_count = df_raw2["伝票日付"].isna().sum()
print(f"✅ df_raw2 NaT 含む行数: {nat_count}")

if nat_count > 0:
    print("✅ df_raw2 NaT 含む行サンプル:")
    print(df_raw2[df_raw2["伝票日付"].isna()].head())


✅ df_raw1 だけにある品名:
set()

✅ df_raw2 だけにある品名:
{'混合廃棄物(羽毛)', 'ﾃﾚﾋﾞ(15型以下)', '混合廃棄物（がれき）', '乾電池'}

✅ df_raw1 だけにある日付:
set()

✅ df_raw2 だけにある日付:
{Timestamp('2024-12-27 00:00:00'), Timestamp('2025-01-04 00:00:00'), Timestamp('2024-06-09 00:00:00'), Timestamp('2024-08-13 00:00:00'), Timestamp('2025-03-09 00:00:00')}

✅ 品名別 件数差分（上位20件）:
                 df_raw1_件数  df_raw2_件数  df_raw1_正味重量  df_raw2_正味重量  \
品名                                                                    
混合廃棄物A              18166.0       18498    11966150.0    12045150.0   
ﾏｯﾄﾚｽ(ｼﾝｸﾞﾙ)          219.0         469       15360.0       17840.0   
混合廃棄物（羽毛物）            147.0         371       28720.0       30960.0   
ﾀﾞﾝﾎﾞｰﾙ               693.0         786       99480.0      100460.0   
GC 軽鉄･ｽﾁｰﾙ類          3384.0        3465     2305330.0     2390240.0   
選別                   3035.0        3109     1363100.0     1368280.0   
混合廃棄物B               5590.0        5653     3396170.0     3421350.0   
軽量物系A(発泡ｽﾁﾛｰﾙ)        365.0  

In [ ]:
import matplotlib.pyplot as plt

# --- df_raw1 用の集計 ---
df1_a = df_raw1[df_raw1["品名"] == "混合廃棄物A"]
df1_a_daily = df1_a.groupby("伝票日付").agg(
    件数=("正味重量", "count"),
    正味重量=("正味重量", "sum")
).sort_index()

# --- df_raw2 用の集計 ---
df2_a = df_raw2[df_raw2["品名"] == "混合廃棄物A"]
df2_a_daily = df2_a.groupby("伝票日付").agg(
    件数=("正味重量", "count"),
    正味重量=("正味重量", "sum")
).sort_index()

# --- プロット ---
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# 件数グラフ
axes[0].plot(df1_a_daily.index, df1_a_daily["件数"], label="df_raw1", linestyle="-", marker="o")
axes[0].plot(df2_a_daily.index, df2_a_daily["件数"], label="df_raw2", linestyle="-", marker="x")
axes[0].set_ylabel("件数")
axes[0].legend()
axes[0].set_title("混合廃棄物A 日別 件数推移")

# 正味重量グラフ
axes[1].plot(df1_a_daily.index, df1_a_daily["正味重量"], label="df_raw1", linestyle="-", marker="o")
axes[1].plot(df2_a_daily.index, df2_a_daily["正味重量"], label="df_raw2", linestyle="-", marker="x")
axes[1].set_ylabel("正味重量 (kg)")
axes[1].legend()
axes[1].set_title("混合廃棄物A 日別 正味重量推移")

plt.xlabel("伝票日付")
plt.tight_layout()
plt.show()


In [ ]:
# --- 必要なパッケージ ---
import pandas as pd
import matplotlib.pyplot as plt

# --- ① df_raw2 から 混合廃棄物Aの日付別 件数集計 ---
df2_a_daily = (
    df_raw2[df_raw2["品名"] == "混合廃棄物A"]
    .groupby("伝票日付")["正味重量"]
    .agg(件数="count", 正味重量="sum")
)

# --- ② 異常な日（件数が100件以上の日）抽出 ---
abnormal_dates = df2_a_daily[df2_a_daily["件数"] > 100].index.tolist()

print("✅ 異常件数日の候補:", abnormal_dates)

# --- ③ 異常日のデータ抽出 ---
df_abnormal = df_raw2[df_raw2["伝票日付"].isin(abnormal_dates)]

# --- ④ 品名別 件数・正味重量 集計 ---
abnormal_summary = df_abnormal.groupby("品名")["正味重量"].agg(
    件数="count", 合計正味重量="sum"
).sort_values(by="件数", ascending=False)

# --- ⑤ 上位20件表示 ---
print("✅ 異常日での 品名別 件数・正味重量（上位20件）:")
print(abnormal_summary.head(20))
